# Week 5 - Vector Space Model (VSM) and Topic Modeling

Over the next weeks, we are going to re-implement Sherin's algorithm and apply it to the text data we've been working on last week! Here's our roadmap:

**Week 6 - vectorization and linear algebra**
6. Dampen: weight the frequency of words (1 + log[count])
7. Scale: Normalize weighted frequency of words
8. Direction: compute deviation vectors

**Week 7 - Clustering**
9. apply different unsupervised machine learning algorithms
    * figure out how many clusters we want to keep
    * inspect the results of the clustering algorithm

**Week 8 - Visualizing the results**
10. create visualizations to compare documents

# WEEK 5 - DATA CLEANING

## Step 1 - Data Retrieval

In [1]:
# using glob, find all the text files in the "Papers" folder
import glob

files = glob.glob('./Papers/*.txt')
print(files)

['./Papers/paper12.txt', './Papers/paper5.txt', './Papers/paper4.txt', './Papers/paper13.txt', './Papers/paper11.txt', './Papers/paper6.txt', './Papers/paper7.txt', './Papers/paper10.txt', './Papers/paper14.txt', './Papers/paper3.txt', './Papers/paper2.txt', './Papers/paper15.txt', './Papers/paper0.txt', './Papers/paper1.txt', './Papers/paper16.txt', './Papers/paper9.txt', './Papers/paper8.txt']


In [2]:
# get all the data from the text files into the "documents" list
# P.S. make sure you use the 'utf-8' encoding
documents = []

for filename in files: 
    with open (filename, "r", encoding='utf-8') as f:
        documents.append(f.read())

In [3]:
# print the first 1000 characters of the first document to see what it 
# looks like (we'll use this as a sanity check below)
documents[0][:100]

'103\n\n\x0cepistemic network analysis and topic modeling for chat\ndata from collaborative learning enviro'

## Step 2 - Data Cleaning

In [4]:
# only select the text that's between the first occurence of the 
# the word "abstract" and the last occurence of the word "reference"
# Optional: print the length of the string before and after, as a 
# sanity check
# HINT: https://stackoverflow.com/questions/14496006/finding-last-occurrence-of-substring-in-string-replacing-that
# read more about rfind: https://www.tutorialspoint.com/python/string_rfind.htm

for i,doc in enumerate(documents):
    print(len(documents[i]), end=' ')
    # only keep the text after the abstract
    doc = doc[doc.index('abstract'):doc.rfind('reference')]
    # save the result
    documents[i] = doc
    # print the length of the resulting string
    print(len(documents[i]))
    
# one liner:
# documents = [doc[doc.index('abstract'):doc.rfind('reference')] for doc in documents]

40387 34778
37214 32762
44037 40032
45258 42251
32277 28206
47851 41302
42617 35102
49177 42621
40655 32734
47377 42978
46761 42253
31574 28134
50043 39318
41110 35514
42046 37649
47845 44059
45724 39947


In [5]:
# replace carriage returns (i.e., "\n") with a white space
# check that the result looks okay by printing the 
# first 1000 characters of the 1st doc:

documents = [doc.replace('\n', ' ') for doc in documents]
print(documents[0][:1000])

abstract this study investigates a possible way to analyze chat data from collaborative learning environments using epistemic network analysis and topic modeling. a 300-topic general topic model built from tasa (touchstone applied science associates) corpus was used in this study. 300 topic scores for each of the 15,670 utterances in our chat data were computed. seven relevant topics were selected based on the total document scores. while the aggregated topic scores had some power in predicting students’ learning, using epistemic network analysis enables assessing the data from a different angle. the results showed that the topic score based epistemic networks between low gain students and high gain students were significantly different (𝑡 = 2.00). overall, the results suggest these two analytical approaches provide complementary information and afford new insights into the processes related to successful collaborative interactions.  keywords chat; collaborative learning; topic modelin

In [6]:
# replace the punctation below by a white space
# check that the result looks okay 
# (e.g., by print the first 1000 characters of the 1st doc)

punctuation = ['.', '...', '!', '#', '"', '%', '$', "'", '&', ')', 
               '(', '+', '*', '-', ',', '/', '.', ';', ':', '=', 
               '<', '?', '>', '@', '",', '".', '[', ']', '\\', ',',
               '_', '^', '`', '{', '}', '|', '~', '−', '”', '“', '’']


# remove ponctuation
for i,doc in enumerate(documents): 
    for punc in punctuation: 
        doc = doc.replace(punc, ' ')
    documents[i] = doc
    
print(documents[0][:1000])

abstract this study investigates a possible way to analyze chat data from collaborative learning environments using epistemic network analysis and topic modeling  a 300 topic general topic model built from tasa  touchstone applied science associates  corpus was used in this study  300 topic scores for each of the 15 670 utterances in our chat data were computed  seven relevant topics were selected based on the total document scores  while the aggregated topic scores had some power in predicting students  learning  using epistemic network analysis enables assessing the data from a different angle  the results showed that the topic score based epistemic networks between low gain students and high gain students were significantly different  𝑡   2 00   overall  the results suggest these two analytical approaches provide complementary information and afford new insights into the processes related to successful collaborative interactions   keywords chat  collaborative learning  topic modelin

In [7]:
# remove numbers by either a white space or the word "number"
# again, print the first 1000 characters of the first document
# to check that you're doing the right thing
for i,doc in enumerate(documents): 
    for num in range(10):
        doc = doc.replace(str(num), '')
    documents[i] = doc

print(documents[1][:1000])

abstract there is a critical need to develop new educational technology applications that analyze the data collected by universities to ensure that students graduate in a timely fashion   to  years   and they are well prepared for jobs in their respective fields of study  in this paper  we present a novel approach for analyzing historical educational records from a large  public university to perform next term grade prediction  i e   to estimate the grades that a student will get in a course that he she will enroll in the next term  accurate next term grade prediction holds the promise for better student degree planning  personalized advising and automated interventions to ensure that students stay on track in their chosen degree program and graduate on time  we present a factorization based approach called matrix factorization with temporal course wise influence that incorporates course wise influence effects and temporal effects for grade prediction  in this model  students and cours

In [8]:
# Remove the stop words below from our documents
# print the first 1000 characters of the first document
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
              'ourselves', 'you', 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his', 'himself', 'she', 
              'her', 'hers', 'herself', 'it', 'its', 'itself', 
              'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 
              'be', 'been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
              'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 
              'between', 'into', 'through', 'during', 'before', 
              'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 
              'further', 'then', 'once', 'here', 'there', 'when', 
              'where', 'why', 'how', 'all', 'any', 'both', 'each', 
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 
              'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
              'too', 'very', 's', 't', 'can', 'will', 
              'just', 'don', 'should', 'now']


# remove stop words
for i,doc in enumerate(documents):
    for stop_word in stop_words:
        doc = doc.replace(' ' + stop_word + ' ', ' ')
    documents[i] = doc

print(documents[0][:1000])

abstract study investigates possible way analyze chat data collaborative learning environments using epistemic network analysis topic modeling   topic general topic model built tasa  touchstone applied science associates  corpus used study   topic scores   utterances chat data computed  seven relevant topics selected based total document scores  aggregated topic scores power predicting students  learning  using epistemic network analysis enables assessing data different angle  results showed topic score based epistemic networks low gain students high gain students significantly different  𝑡       overall  results suggest two analytical approaches provide complementary information afford new insights processes related successful collaborative interactions   keywords chat  collaborative learning  topic modeling  epistemic network analysis    introduction collaborative learning special form learning interaction affords opportunities groups students combine cognitive resources synchronousl

In [9]:
# remove words with one and two characters (e.g., 'd', 'er', etc.)
# print the first 1000 characters of the first document

for i,doc in enumerate(documents):  
    doc = [x for x in doc.split() if len(x) > 2]
    doc = " ".join(doc)
    documents[i] = doc

print(documents[0][:1000])

abstract study investigates possible way analyze chat data collaborative learning environments using epistemic network analysis topic modeling topic general topic model built tasa touchstone applied science associates corpus used study topic scores utterances chat data computed seven relevant topics selected based total document scores aggregated topic scores power predicting students learning using epistemic network analysis enables assessing data different angle results showed topic score based epistemic networks low gain students high gain students significantly different overall results suggest two analytical approaches provide complementary information afford new insights processes related successful collaborative interactions keywords chat collaborative learning topic modeling epistemic network analysis introduction collaborative learning special form learning interaction affords opportunities groups students combine cognitive resources synchronously asynchronously participate ta


### Putting it all together

In [10]:
# package all of your work above into a function that cleans a given document

def clean_list_of_documents(documents):
    
    cleaned_docs = []

    for i,doc in enumerate(documents):
        # only keep the text after the abstract
        doc = doc[doc.index('abstract'):]
        # only keep the text before the references
        doc = doc[:doc.rfind('reference')]
        # replace return carriage with white space
        doc = doc.replace('\n', ' ')
        # remove ponctuation
        for punc in punctuation: 
            doc = doc.replace(punc, ' ')
        # remove numbers
        for i in range(10):
            doc = doc.replace(str(i), ' ')
        # remove stop words
        for stop_word in stop_words:
            doc = doc.replace(' ' + stop_word + ' ', ' ')
        # remove single characters and stem the words 
        doc = [x for x in doc.split() if len(x) > 2]
        doc = " ".join(doc)
        # save the result to our list of documents
        cleaned_docs.append(doc)
        
    return cleaned_docs

In [11]:
# reimport your raw data
documents = []

for filename in files: 
    with open (filename, "r", encoding='utf-8') as f:
        documents.append(f.read())
        
# clean your files using the function above
docs = clean_list_of_documents(documents)

# print the first 1000 characters of the first document
print(docs[0][:1000])

abstract study investigates possible way analyze chat data collaborative learning environments using epistemic network analysis topic modeling topic general topic model built tasa touchstone applied science associates corpus used study topic scores utterances chat data computed seven relevant topics selected based total document scores aggregated topic scores power predicting students learning using epistemic network analysis enables assessing data different angle results showed topic score based epistemic networks low gain students high gain students significantly different overall results suggest two analytical approaches provide complementary information afford new insights processes related successful collaborative interactions keywords chat collaborative learning topic modeling epistemic network analysis introduction collaborative learning special form learning interaction affords opportunities groups students combine cognitive resources synchronously asynchronously participate ta

## Step 3 - Build your list of vocabulary

This list of words (i.e., the vocabulary) is going to become the columns of your matrix.

In [12]:
import math
import numpy as np

In [13]:
# create a function that takes in a list of documents
# and returns a set of unique words. Make sure that you
# sort the list alphabetically before returning it. 

def get_vocabulary(docs):
    voc = []
    for doc in docs:
        for word in doc.split():
            if word not in voc: 
                voc.append(word)
    voc = list(set(voc))
    voc.sort()
    return voc

# Then print the length of your vocabulary (it should be 
# around 5500 words)
vocabulary = get_vocabulary(docs)
print(len(vocabulary))

5676


## Step 4 - transform your documents in to 100-words chunks

In [14]:
# create a function that takes in a list of documents
# and returns a list of 100-words chunk 
# (with a 25 words overlap between them)
# Optional: add two arguments, one for the number of words
# in each chunk, and one for the overlap

def flatten_and_overlap(docs, window_size=100, overlap=25):
    
    # create the list of overlapping documents
    new_list_of_documents = []
    
    # flatten everything into one string
    flat = ""
    for doc in docs:
        flat += doc
    
    # split into words
    flat = flat.split()

    # create chunks of 100 words
    high = window_size
    while high < len(flat):
        low = high - window_size
        new_list_of_documents.append(flat[low:high])
        high += overlap
    return new_list_of_documents

chunks = flatten_and_overlap(docs)

In [15]:
# create a for loop to double check that each chunk has 
# a length of 100
# Optional: use assert to do this check
for chunk in chunks: 
    assert(len(chunk) == 100)

# WEEK 6 - VECTOR MANIPULATION

## Step 5 - Create a word by document matrix

In [16]:
# 1) create an empty dataframe using pandas
# the number of rows should be the number of chunks we have
# the number of columns should be size of the vocabulary

import pandas as pd

df = pd.DataFrame(0, index=np.arange(len(chunks)), columns=vocabulary)

In [17]:
# 2) fill out the dataframe with the count of words for each chunk
# (use two for loops to iterate through the documents and the vocabulary)

for i, chunk in enumerate(chunks):
    for word in chunk:
        if word in df.columns:
            df.loc[i, word] += 1

    if i % 100 ==0:
        print(i,end=' ')

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 

In [18]:
# 3) Sanity check: make sure that your counts are correct
# (e.g., if you know that a words appears often in a chunk, check that
# the number is also high in your dataframe; and vice-versa for low counts)

x = 0
c = 8
word = 'wandering'
for i in range(0,len(chunks)):
    if chunks[i].count(word) > c:
        x = i

print(df.at[x,word] == chunks[x].count(word))

True


In [19]:
# 4) Putting it together: create a function that takes a list of chunk
# and a vocabulary as arguments, and returns a dataframe with the counts
# of words: 

def df_count(chunks,vocabulary):
    
    df = pd.DataFrame(0, index=np.arange(len(chunks)), columns=vocabulary)
                      
    for i, chunk in enumerate(chunks):
        for word in chunk:
            if word in df.columns:
                df.loc[i, word] += 1
    
        if i % 100 ==0:
            print(i,end=' ')                  
    
    return df

# call the function and check that the resulting dataframe is correct
df2 = df_count(chunks, vocabulary)

x = 0
c = 8
word = 'wandering'
for i in range(0,len(chunks)):
    if chunks[i].count(word) > c:
        x = i

print(df2.at[x,word] == chunks[x].count(word))

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 True


## Step 6 - Weight word frequency

In [20]:
# 5) create a function that adds one to the current cell and takes its log
# IF the value in the cell is not zero

df_weight = pd.DataFrame(0, index=np.arange(len(chunks)), columns=vocabulary)

for i in range(0,df2.shape[0]):
    for j in range(0,df2.shape[1]):
        if df2.iloc[i,j] != 0:
            df_weight.iloc[i,j] = 1 + np.log(df2.iloc[i,j])
    
    if i % 100 ==0:
            print(i,end=' ')

0 

KeyboardInterrupt: 

In [21]:
# 6) use the "applymap" function of the dataframe to apply the function 
# above to each cell of the table

df_weight = df2.applymap(lambda x: 1 + np.log(x) if x != 0 else x)


In [22]:
# 7) check that the numbers in the resulting matrix look accurate;
# print the value before and after applying the function above

x = 0
c = 8
word = 'wandering'
for i in range(0,len(chunks)):
    if chunks[i].count(word) > c:
        x = i

print(df2.at[x,word])
print(1+np.log(df2.at[x,word]))
print(df_weight.at[x,word])

9
3.1972245773362196
3.1972245773362196


## Step 7 - Matrix normalization

In [23]:
# 8) look at the image below; why do you think that we need to normalize our 
# data before clustering in this particular case? 

print('It appears that the two clusters of data are operating on different scales and hence the two clusters appear on two distinct regions on the graph. In order for us to be able to compare and do meaningful analysis of the two clusters, we need to normalize to get the two clusters to operate on the same scale.')

It appears that the two clusters of data are operating on different scales and hence the two clusters appear on two distinct regions on the graph. In order for us to be able to compare and do meaningful analysis of the two clusters, we need to normalize to get the two clusters to operate on the same scale.


<img src="https://i.stack.imgur.com/N2unM.png" />

In general, it's common practice to normalize your data before clustering - so that variables are comparable.

In [24]:
# 9) describe how the min-max normalization works:

print('The min-max normalization removes the smallest value of the data from all data and then downscale the values of all data by the range of the data. In other words, the normalized data will have values from 0 to 1.')

The min-max normalization removes the smallest value of the data from all data and then downscale the values of all data by the range of the data. In other words, the normalized data will have values from 0 to 1.


<img src="https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/media/aml-normalization-minmax.png" />

In [25]:
# 10) describe how normalizing using a z-score works:

print('The z-score normalization removes the mean value of the data from all data and then downscale the values of all data by the standard deviation of the data. In other words, the normalized data will have values that represent the number of units of standard deviation from 0.')

The z-score normalization removes the mean value of the data from all data and then downscale the values of all data by the standard deviation of the data. In other words, the normalized data will have values that represent the number of units of standard deviation from 0.


<img src="https://cdn-images-1.medium.com/max/1600/1*13XKCXQc7eabfZbRzkvGvA.gif"/>

In [26]:
# 11) describe how normalizing to unit norm works

print('Normalizing to unit norm first determines the normal vector (i.e. perpendicular vector) of an original vector, and then normalize the normal vector by changing the length of the normal vector to 1 unit length.')

Normalizing to unit norm first determines the normal vector (i.e. perpendicular vector) of an original vector, and then normalize the normal vector by changing the length of the normal vector to 1 unit length.


Resources: 
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer
* http://mathworld.wolfram.com/NormalVector.html

We are going to work with some pre-made normalization functions from sklearn (feel free to skim this page):
* https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

In [27]:
# 12) since we are working with vectors, apply the Normalizer from 
# sklearn.preprocessing to our dataframe. Print a few values 
# before and after to make sure you've applied the normalization
from sklearn.preprocessing import Normalizer

transformer = Normalizer().fit(df_weight)
df_norm = pd.DataFrame(transformer.transform(df_weight),columns=vocabulary)

x = 0
c = 0
for i in range(0,len(chunks)):
    if df_weight.iloc[i,0] > c:
        x = i

print(df_weight.iat[x,0])
print(df_norm.iat[x,0])


1.6931471805599454
0.1492897240156656


In [28]:
# 13) create a function that takes a dataframe as argument and where a second
# argument is the type of normalization (MinMaxScaler, Normalizer, StandardScaler)
# and returns the normalized dataframe
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

def normalize_df (df, type_of_norm):
    
    if type_of_norm == 'MinMaxScalar':
        transformer = MinMaxScalar().fit(df)
        df_norm = pd.DataFrame(transformer.transform(df),columns=vocabulary)
        
    elif type_of_norm == 'Normalizer':
        transformer = Normalizer().fit(df)
        df_norm = pd.DataFrame(transformer.transform(df),columns=vocabulary)
        
    elif type_of_norm == 'StandardScalar':
        transformer = StandardScalar().fit(df)
        df_norm = pd.DataFrame(transformer.transform(df),columns=vocabulary)
    
    return df_norm

## Step 8 - Deviation Vectors

<img src="https://www.dropbox.com/s/9f73r7pk7bi7vh9/deviation_vectors.png?dl=1" />

In [29]:
# 14) compute the sum of the vectors

import numpy as np
avg_vec = np.zeros(len(vocabulary))

for row in df_norm.rows:
    avg_vec = avg_vec + np.array(row)

avg_vec

array([4.06685612, 4.06685612, 4.06685612, ..., 4.06685612, 4.06685612,
       4.06685612])

In [30]:
# 15) normalize the vector (find its average)

avg_vec = avg_vec / np.linalg.norm(avg_vec)
avg_vec

array([0.0132733, 0.0132733, 0.0132733, ..., 0.0132733, 0.0132733,
       0.0132733])

In [31]:
# 16) take each vector and subtract its components along v_avg

dev_vec = np.zeros(len(vocabulary))
dot_product = 0
dev_matrix = []

for row in df_norm.rows:
    dot_product = np.dot(row, avg_vec)
    dev_vec = np.array(row) - dot_product*avg_vec
    dev_vec = dev_vec / np.linalg.norm(dev_vec)
    
    dev_matrix.append(dev_vec)
    dev_vec=np.zeros(len(vocabulary))

dev_matrix = pd.DataFrame(dev_matrix, columns=vocabulary)
dev_matrix = dev_matrix.fillna(0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [32]:
# 17) put the code above in a function that takes in a dataframe as an argument
# and computes deviation vectors of each row (=chunk)

def dev_func(df):
    dev_vec = np.zeros(len(vocabulary))
    dot_product = 0
    dev_matrix = []

    for row in df.rows:
        dot_product = np.dot(row, avg_vec)
        dev_vec = np.array(row) - dot_product*avg_vec
        dev_vec = dev_vec / np.linalg.norm(dev_vec)
    
        dev_matrix.append(dev_vec)
        dev_vec=np.zeros(len(vocabulary))

    dev_matrix = pd.DataFrame(dev_matrix, columns=vocabulary)
    dev_matrix = dev_matrix.fillna(0)

    return dev_matrix
    

# WEEK 7 - CLUSTERING

## Step 9 - Clustering

## Step 10 - Visualizing the results

## Final Step - Putting it all together: 

In [ ]:
# in python code, our goal is to recreate the steps above as functions
# so that we can just one line to run topic modeling on a list of 
# documents: 
def ExtractTopicsVSM(documents, numTopics):
    ''' this functions takes in a list of documents (strings), 
        runs topic modeling (as implemented by Sherin, 2013)
        and returns the clustering results, the matrix used 
        for clustering a visualization '''
    
    # step 2: clean up the documents
    documents = clean_list_of_documents(documents)
    
    # step 3: let's build the vocabulary of these docs
    vocabulary = get_vocabulary(documents)
    
    # step 4: we build our list of 100-words overlapping fragments
    documents = flatten_and_overlap(documents)
    
    # step 5: we convert the chunks into a matrix
    matrix = docs_by_words_matrix(documents, vocabulary)
    
    # step 6: we weight the frequency of words (count = 1 + log(count))
    matrix = one_plus_log_mat(matrix, documents, vocabulary)
    
    # step 7: we normalize the matrix
    matrix = normalize(matrix)
    
    # step 8: we compute deviation vectors
    matrix = transform_deviation_vectors(matrix, documents)
    
    # step 9: we apply a clustering algorithm to find topics
    results_clustering = cluster_matrix(matrix)
    
    # step 10: we create a visualization of the topics
    visualization = visualize_clusters(results_clustering, vocabulary)
    
    # finally, we return the clustering results, the matrix, and a visualization
    return results_clustering, matrix, visualization